# Rag Simple

## Imports

In [ ]:
from typing import TypedDict, List
from langgraph.graph import StateGraph, END
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate


class RAGState(TypedDict):
    """Shared state"""
    question: str
    documents: List[Document]
    retrieved_docs: List[Document]
    answer: str
    error: str | None


## Configuration

In [ ]:
# Modèle d'embeddings simple et gratuit
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# LLM pour la génération
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Template de prompt simple
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "Tu es un assistant qui répond aux questions en utilisant UNIQUEMENT les documents fournis. Si l'information n'est pas dans les documents, dis-le clairement."),
    ("user", """Documents:
{context}

Question: {question}

Réponds de manière concise et précise.""")
])


# Nodes

In [ ]:
def chunk_documents(state: RAGState) -> RAGState:
    """Découpe les documents en chunks"""
    print("Chunking des documents...")
    
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=50,
        separators=["\n\n", "\n", ".", " ", ""]
    )
    
    chunks = splitter.split_documents(state["documents"])
    state["documents"] = chunks
    print(f"{len(chunks)} chunks créés")
    return state


def create_vectorstore(state: RAGState) -> RAGState:
    """Crée la base vectorielle et indexe les documents"""
    print("Création de la base vectorielle...")
    
    # Création de l'index FAISS
    vectorstore = FAISS.from_documents(
        state["documents"],
        embeddings
    )
    
    # Stockage dans l'état (en production, utiliser une DB persistante)
    state["vectorstore"] = vectorstore
    print("Base vectorielle créée")
    return state


def retrieve_documents(state: RAGState) -> RAGState:
    """Recherche les documents pertinents"""
    print(f"Recherche pour: '{state['question']}'")
    
    vectorstore = state.get("vectorstore")
    if not vectorstore:
        state["error"] = "Base vectorielle non initialisée"
        return state
    
    # Recherche par similarité (top 3)
    docs = vectorstore.similarity_search(
        state["question"],
        k=3
    )
    
    state["retrieved_docs"] = docs
    print(f"{len(docs)} documents récupérés")
    return state


def generate_answer(state: RAGState) -> RAGState:
    """Génère la réponse avec le LLM"""
    print("Génération de la réponse...")
    
    if not state["retrieved_docs"]:
        state["answer"] = "Je n'ai pas trouvé d'informations pertinentes dans les documents."
        return state
    
    # Prépare le contexte
    context = "\n\n".join([
        f"Document {i+1}:\n{doc.page_content}"
        for i, doc in enumerate(state["retrieved_docs"])
    ])
    
    # Génère la réponse
    chain = prompt_template | llm
    response = chain.invoke({
        "context": context,
        "question": state["question"]
    })
    
    state["answer"] = response.content
    print("Réponse générée")
    return state

# Graph

In [ ]:
def create_simple_rag_graph():
    """Crée le graphe RAG simple"""
    
    workflow = StateGraph(RAGState)
    
    # Ajout des nœuds
    workflow.add_node("chunk", chunk_documents)
    workflow.add_node("vectorize", create_vectorstore)
    workflow.add_node("retrieve", retrieve_documents)
    workflow.add_node("generate", generate_answer)
    
    # Définition du flux
    workflow.set_entry_point("chunk")
    workflow.add_edge("chunk", "vectorize")
    workflow.add_edge("vectorize", "retrieve")
    workflow.add_edge("retrieve", "generate")
    workflow.add_edge("generate", END)
    
    return workflow.compile()

## Test du workflow 

In [ ]:
if __name__ == "__main__":
    # Exemple de documents
    sample_docs = [
        Document(
            page_content="""
            Article 1101 du Code civil :
            Le contrat est un accord de volontés entre deux ou plusieurs personnes destiné à créer, modifier, transmettre ou éteindre des obligations.
            """,
            metadata={
                "source": "code_civil.txt",
                "article": "1101",
                "section": "Des contrats",
                "thème": "Définition du contrat"
            }
        ),
        Document(
            page_content="""
            Article 1102 du Code civil :
            Le contrat est synallagmatique ou unilateral selon que les contractants s'obligent réciproquement les uns envers les autres ou que l'un d'eux s'engage envers l'autre sans que celui-ci s'engage.
            """,
            metadata={
                "source": "code_civil.txt",
                "article": "1102",
                "section": "Des contrats",
                "thème": "Types de contrats"
            }
        ),
        Document(
            page_content="""
            Article 1103 du Code civil :
            Les contrats légalement formés tiennent lieu de loi à ceux qui les ont faits. Ils ne peuvent être révoqués que de leur consentement mutuel, ou pour les causes que la loi autorise.
            """,
            metadata={
                "source": "code_civil.txt",
                "article": "1103",
                "section": "Des contrats",
                "thème": "Force obligatoire du contrat"
            }
        ),
        Document(
            page_content="""
            Article 1104 du Code civil :
            Les contrats doivent être négociés, formés et exécutés de bonne foi. Cette disposition est d'ordre public.
            """,
            metadata={
                "source": "code_civil.txt",
                "article": "1104",
                "section": "Des contrats",
                "thème": "Bonne foi"
            }
        ),
        Document(
            page_content="""
            Article 1231-1 du Code civil :
            L'obligation est une contrainte juridique en vertu de laquelle une personne, le débiteur, est tenue d'accomplir une prestation au profit d'une autre, le créancier.
            """,
            metadata={
                "source": "code_civil.txt",
                "article": "1231-1",
                "section": "Des obligations",
                "thème": "Définition de l'obligation"
            }
        )
    ]
    
    # Création du graphe
    app = create_simple_rag_graph()
    
    # État initial (avec documents pré-chargés)
    initial_state = {
        "question": "Quelle est la politique de remboursement ?",
        "documents": sample_docs,
        "retrieved_docs": [],
        "answer": "",
        "error": None
    }
    
    # Exécution
    print("\n" + "="*60)
    print("RAG SIMPLE - DÉMONSTRATION")
    print("="*60 + "\n")
    
    result = app.invoke(initial_state)
    
    print("\n" + "="*60)
    print("RÉSULTAT")
    print("="*60)
    print(f"\nQuestion: {result['question']}")
    print(f"\nRéponse:\n{result['answer']}")
    print(f"\nDocuments utilisés: {len(result['retrieved_docs'])}")
    
    # Test d'une autre question
    print("\n\n" + "="*60)
    print("NOUVELLE QUESTION")
    print("="*60 + "\n")
    
    # Pour une nouvelle question, on réutilise la vectorstore
    new_state = {
        "question": "Quelles sont les garanties du pack Premium ?",
        "documents": [],
        "retrieved_docs": [],
        "answer": "",
        "error": None,
        "vectorstore": result.get("vectorstore")  # Réutilisation
    }
    
    # On skip les étapes de chunking et vectorization
    retrieval_workflow = StateGraph(RAGState)
    retrieval_workflow.add_node("retrieve", retrieve_documents)
    retrieval_workflow.add_node("generate", generate_answer)
    retrieval_workflow.set_entry_point("retrieve")
    retrieval_workflow.add_edge("retrieve", "generate")
    retrieval_workflow.add_edge("generate", END)
    
    retrieval_app = retrieval_workflow.compile()
    result2 = retrieval_app.invoke(new_state)
    
    print(f"Question: {result2['question']}")
    print(f"\nRéponse:\n{result2['answer']}")
